In [ ]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json


--2021-05-30 13:25:40--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.05s   

2021-05-30 13:25:40 (107 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [ ]:
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])


import numpy as np
labels = np.array(labels)

In [ ]:
# Chia features tập train
training_sentences = sentences[0:training_size]
# FIXME: Chia features tập test
testing_sentences = sentences[training_size:]
# FIXME: Chia labels tập train
training_labels =  labels[0:training_size]
# FIXME: Chia labels tập test
testing_labels =  labels[training_size:]

In [ ]:
len (training_labels)

20000

In [ ]:
len (testing_labels)

6709

In [ ]:
# FIXME: Tạo tokenizer
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)

# FIXME: Đưa từ vào tokenizer để tạo từ điển

tokenizer.fit_on_texts(training_sentences)

# FIXME: Biến các câu train thành sequences

training_sequences = tokenizer.texts_to_sequences(training_sentences)

# FIXME: Padding cho các train sequences

training_padded = pad_sequences(training_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

# FIXME: Biến các câu test thành sequences và padding tương ứng

test_sequences = tokenizer.texts_to_sequences(testing_sentences)

test_padded = pad_sequences(test_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

In [ ]:
training_padded.shape

(20000, 100)

In [ ]:
test_padded.shape

(6709, 38)

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling1D

In [ ]:
# Xây dựng model

# FIXME: Tạo model Sequential

model = Sequential()

# FIXME: Thêm một lớp Embedding

model.add(Embedding(vocab_size,embedding_dim,input_length=max_length))

model.add(GlobalAveragePooling1D())

# FIXME: Thêm một lớp Dense, chú ý có activation

model.add(Dense(24,activation='relu'))

# FIXME: Thêm một lớp Dense cuối, chú ý có activation

model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# FIXME: Compile Optimizer và Loss function

model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

In [ ]:
num_epochs=30

In [ ]:
# FIXME: Tiến hành training

model.fit(training_padded,training_labels,epochs=num_epochs,validation_data=(test_padded,testing_labels))

Epoch 1/30
625/625 [==============================] - 3s 4ms/step - loss: 0.6682 - accuracy: 0.5818 - val_loss: 0.5964 - val_accuracy: 0.7626
Epoch 2/30
625/625 [==============================] - 3s 4ms/step - loss: 0.4401 - accuracy: 0.8259 - val_loss: 0.3867 - val_accuracy: 0.8384
Epoch 3/30
625/625 [==============================] - 3s 4ms/step - loss: 0.3126 - accuracy: 0.8755 - val_loss: 0.3566 - val_accuracy: 0.8484
Epoch 4/30
625/625 [==============================] - 3s 4ms/step - loss: 0.2608 - accuracy: 0.8959 - val_loss: 0.3417 - val_accuracy: 0.8572
Epoch 5/30
625/625 [==============================] - 3s 4ms/step - loss: 0.2238 - accuracy: 0.9129 - val_loss: 0.3528 - val_accuracy: 0.8529
Epoch 6/30
625/625 [==============================] - 3s 4ms/step - loss: 0.1962 - accuracy: 0.9242 - val_loss: 0.3589 - val_accuracy: 0.8502
Epoch 7/30
625/625 [==============================] - 3s 4ms/step - loss: 0.1744 - accuracy: 0.9351 - val_loss: 0.3667 - val_accuracy: 0.8538
Epoch 

In [ ]:
# FIXME: Tiến hành dự đoán

In [ ]:
test_sen = ["Everyone seems normal until you get to know them."]

# FIXME: Chuyển câu thành Sequences
test_sen_sequences=tokenizer.texts_to_sequences(test_sen)

# FIXME: padding cho câu này
padded_test_sen_sequences=pad_sequences(test_sen_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)
# FIXME: Tiến hành dự đoán

model.predict(padded_test_sen_sequences)

array([[0.9998473]], dtype=float32)